In [173]:
#import needed packages
import pandas as pd
import numpy as np

In [174]:
#read the raw data
data = pd.read_csv('./datasets/multipleChoiceResponses.csv', encoding='ISO-8859-1', low_memory=False)
#get currency conversion rates
conv = pd.read_csv('./datasets/conversionRates.csv', encoding='ISO-8859-1', low_memory=False)
#get continents
cont = pd.read_csv('./datasets/ContinentMapping.csv', low_memory=False)

In [175]:
#see how large  the survey data is 
data.shape

(16716, 228)

In [176]:
#subset to only workers who are working full time
ft_data = data.loc[data['EmploymentStatus']=='Employed full-time'  ]

#another flag for people who are not working
ft_data = ft_data.loc[ft_data['SalaryChange'] !='I am not currently employed']

#get rid of anyone who doesn't list a job satisfaction rating as it is the dependent variable of interest
final_data = ft_data.loc[ft_data['JobSatisfaction'].notnull() ==True ]

#these removals take us down to 5,619 responses
final_data.shape

(5619, 228)

In [177]:
#convert other currencies to USD so they can rationally be compared 
final_data = pd.merge(final_data, conv, how='left', left_on = 'CompensationCurrency', right_on = 'originCountry')
final_data['CompensationAmount']= final_data['CompensationAmount'].str.replace( ',', '')
final_data['CompensationAmount']= final_data['CompensationAmount'].str.replace( '-', '')
final_data['CompensationAmount']= final_data['CompensationAmount'].str.replace( '-1', '')
final_data['CompensationAmount']= final_data['CompensationAmount'].str.replace( '-99', '')
final_data['CompensationUSD'] =  pd.to_numeric(final_data['CompensationAmount']) * final_data.exchangeRate
#final_data['CompensationUSD'] = final_data.loc[[((final_data['CompensationUSD'] <= 500000) | (final_data['CompensationUSD'].empty() == True))]]

#discovered two bad actors in the data by looking at observations that had both extreme compensation and
#nonsensical results
#final_data.sort_values(by='CompensationUSD', ascending = 0)
final_data = final_data.drop([2623, 1359])

#drop uneeded compensation columns
#'originCountry'
final_data = final_data.drop(['CompensationAmount', 'exchangeRate', 'CompensationCurrency'], axis=1)
#further analysis showed that it is missing for 1/3 of the data and might not be interpreted right so dropping it
#final_data['CompensationUSD'].isnull().sum()
#final_data = final_data.drop('CompensationUSD', axis=1)

In [178]:
#remove all values below 1,000 and caps all salaries at 400K 
final_data.set_value(final_data[final_data['CompensationUSD']<=1000]['CompensationUSD'].index, 'CompensationUSD', value = None)
final_data.set_value(final_data[final_data['CompensationUSD']>=400000]['CompensationUSD'].index, 'CompensationUSD', value = 400000)


,GenderSelect,Country,Age,EmploymentStatus,StudentStatus,LearningDataScience,CodeWriter,CareerSwitcher,CurrentJobTitleSelect,TitleFit,...,JobFactorCompanyFunding,JobFactorImpact,JobFactorRemote,JobFactorIndustry,JobFactorLeaderReputation,JobFactorDiversity,JobFactorPublishingOpportunity,Unnamed: 0,originCountry,CompensationUSD
0,Male,Taiwan,38.0,Employed full-time,NaN,NaN,Yes,NaN,Computer Scientist,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Male,Brazil,46.0,Employed full-time,NaN,NaN,Yes,NaN,Data Scientist,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Male,United States,35.0,Employed full-time,NaN,NaN,Yes,NaN,Computer Scientist,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Female,Australia,43.0,Employed full-time,NaN,NaN,Yes,NaN,Business Analyst,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,AUD,64184.800
4,Male,Russia,33.0,Employed full-time,NaN,NaN,Yes,NaN,Software Developer/Software Engineer,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,RUB,20882.400
5,Male,India,27.0,Employed full-time,NaN,NaN,Yes,NaN,Data Scientist,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,INR,1483.900
6,Male,Taiwan,26.0,Employed full-time,NaN,NaN,Yes,NaN,Software Developer/Software Engineer,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,TWD,36634.400
7,Male,United States,33.0,Employed full-time,NaN,NaN,Yes,NaN,Scientist/Researcher,Perfectly,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,USD,100000.000
8,Male,United States,NaN,Employed full-time,NaN,NaN,Yes,NaN,Software Developer/Software Engineer,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Male,Russia,22.0,Employed full-time,NaN,NaN,Yes,NaN,Data Analyst,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,RUB,10858.848


In [179]:
df_count_avg = final_data.groupby(by='Country')['CompensationUSD'].agg(['mean'])
df_count_avg.columns = ['Average Salary Within Country']
final_data = pd.merge(final_data, df_count_avg, left_on = 'Country', right_index =True, how = 'left')
final_data['Percent Above/Below Average Salary'] = (final_data['CompensationUSD']/final_data['Average Salary Within Country']-1)
#drop raw compensation as it has limited meaning across countries
final_data = final_data.drop('CompensationUSD', axis=1)

In [180]:
#count the number of challenges someone lists (of any kind)
final_data['NumberChallenges'] = final_data.loc[:,'WorkChallengeFrequencyPolitics':'WorkChallengeFrequencyOtherSelect'].count(axis=1)
final_data = final_data.drop('WorkChallengesSelect', axis=1)

#count the number of platforms someone has access to 
final_data['NumberPlatforms'] = final_data.loc[:,'LearningPlatformUsefulnessArxiv':'LearningPlatformUsefulnessYouTube'].count(axis=1)
final_data = final_data.drop('LearningPlatformSelect', axis=1)

#text field that aren't useful for our analysis 
final_data = final_data.drop('WorkDatasetsChallenge', axis=1)
final_data = final_data.drop('WorkDatasets', axis=1)

#not something organizations can act on 
final_data = final_data.drop('PublicDatasetsSelect', axis=1)
final_data = final_data.drop('BlogsPodcastsNewslettersSelect', axis=1)
final_data = final_data.drop('PastJobTitlesSelect', axis=1)

#not easily usable
final_data = final_data.drop('WorkMethodsFrequencySelect3', axis=1)

In [181]:
#Identifies all variables that use the term 'Rarely' for recoding later
rarely_list = []
for col in final_data.select_dtypes(include={'object'}).columns:
    if final_data[col].str.contains('Rarely').any():
        rarely_list.append(col)

In [182]:
#adds additional columsn that need to have recodes done and adds them to the list
WorkTools = [col for col in final_data.columns if 'WorkTools' in col]
WorkMethods  = [col for col in final_data.columns if 'WorkMethods' in col]
WorkChallenge = [col for col in final_data.columns if 'WorkChallenge' in col]

biglist = rarely_list+WorkTools+WorkMethods+WorkChallenge
biglist = list(set(biglist))
biglist.sort()
len(biglist)

109

In [183]:
#recodes everything that is missing to 'Never' since not checking it means roughly the same thing as chekcing 'Never'
final_data[biglist] = final_data[biglist].replace(np.nan, 'Never')

In [184]:
#converts don't knows into 'missing' as don't know doesn't really grant meaningful information
final_data[['WorkProductionFrequency', 'RemoteWork']] = final_data[['WorkProductionFrequency', 'RemoteWork']].replace(
    ["Don't know"], ["Missing"])

#["Don't know", 'Never', 'Rarely', 'Sometimes', 'Often', 'Most of the time']
final_data = final_data.replace(["Don't know"], ["Missing"])

In [185]:
#manually created a mapping countries to continents 
final_data = pd.merge(final_data, cont, how='left', left_on = 'Country', right_on = 'Country')

#dropping country data as it is too sparse to be used outside of a few large countries
final_data = final_data.drop(['Country'], axis=1)

In [186]:
#takes a dataframe and a specified column that is a comma separated list of values
#splits out the lists present in the survey into interpretable dummy variables 
#returns a dataframe with the new data

def CSVListToDummy (df, col):
    full_dummy= pd.DataFrame()
    for i in range(0, df[col].str.split(',', expand=True).shape[1]):
        #Other is common and used across multiple varialbes and is generally useless so dropping
        temp = pd.get_dummies(df[col].str.split(',', expand=True)[i])
        try:
            temp = temp.drop('Other', axis=1)
        except:
            pass
        full_dummy = full_dummy.radd(temp, fill_value  = 0)
        temp = None
    return full_dummy

#create dummy variables for a number of variables 
WorkToolDummy= CSVListToDummy (final_data, 'WorkToolsSelect') 
MLSkillsDummy= CSVListToDummy (final_data, 'MLSkillsSelect')  
HardwareDummy = CSVListToDummy (final_data, 'WorkHardwareSelect')  
WorkAlgoDummy = CSVListToDummy (final_data, 'WorkAlgorithmsSelect')
WorkCodeSharingDummy = CSVListToDummy (final_data, 'WorkCodeSharing')
EmployerTypeDummy = CSVListToDummy (final_data, 'CurrentEmployerType')
MLToolDummy = CSVListToDummy (final_data, 'MLTechniquesSelect')
StorageDummy = CSVListToDummy (final_data, 'WorkDataStorage')
SharingDummy = CSVListToDummy (final_data, 'WorkDataSharing')
SourcingDummy = CSVListToDummy (final_data, 'WorkDataSourcing')
WorkDummy = CSVListToDummy (final_data, 'WorkMethodsSelect') 

#creates 'total count' variables that are sums of all of the dummy variables that are created above
WorkToolCount = pd.DataFrame(WorkToolDummy.sum(axis=1), columns = {'Tools Used At Work Count'})
MLSkillCount = pd.DataFrame(MLSkillsDummy.sum(axis=1), columns = {'ML Skill Count'})
HardwareCount = pd.DataFrame(HardwareDummy.sum(axis=1), columns = {'Hardware Types Used'})
WorkAlgoCount = pd.DataFrame(WorkAlgoDummy.sum(axis=1), columns = {'Number of Algorithims'})
WorkCodeSharingCount = pd.DataFrame(WorkCodeSharingDummy.sum(axis=1), columns = {'Number of Code Sharing Methods'})
MLToolCount = pd.DataFrame(MLToolDummy.sum(axis=1), columns = {'Number of Machine Learning Methods'})
StorageCount = pd.DataFrame(StorageDummy.sum(axis=1), columns = {'Number of Storage Options'})
SharingCount = pd.DataFrame(SharingDummy.sum(axis=1), columns = {'Number of Sharing Options'})
SourcingCount = pd.DataFrame(SourcingDummy.sum(axis=1), columns = {'Number of Sourcing Options'})
WorkCount = pd.DataFrame(WorkDummy.sum(axis=1), columns = {'Number of Work Methods'})

In [187]:
#merge all of the new dummy data back into the original dataframe
final_data = final_data.join(WorkToolDummy).join(MLSkillsDummy).join(WorkToolCount).join(MLSkillCount).join(HardwareDummy).join(HardwareCount).join(WorkAlgoDummy).join(WorkAlgoCount).join(WorkCodeSharingDummy).join(WorkCodeSharingCount).join(EmployerTypeDummy)

In [188]:
#drop the original text columns as they're no longer needed 
final_data = final_data.drop(['WorkToolsSelect','MLSkillsSelect','WorkHardwareSelect', 'WorkAlgorithmsSelect', 'WorkCodeSharing', 'CurrentEmployerType', 'MLTechniquesSelect', 'WorkDataStorage', 'WorkDataSharing', 'WorkDataSourcing', 'WorkMethodsSelect' ], axis=1)

In [189]:
#drop columns that do not have a value for at least 200 values as it could drive overfitting in a population this small
final_data = final_data.dropna(axis=1, how='all', thresh=10)

In [190]:
final_data.shape

(5617, 278)

In [191]:
#convert the rarely-most of the time scales to a 4 point scale. There are many columns so apply to the entire dataframe 
#seemed like a good idea but the model actually performs much worse in practice so getting ride of it
#final_data= final_data.replace( 'Rarely', '1')
#final_data= final_data.replace( 'Sometimes', '2')
#final_data= final_data.replace( 'Often', '3')
#final_data= final_data.replace( 'Most of the time', '4')
#final_data['WorkToolsFrequencyJupyter'].unique()

In [192]:
#cleanse Job Satisfaction so that it is a numberic variable that can be a dependent variable in our Random Forest
final_data['JobSatisfaction'].replace(to_replace = '1 - Highly Dissatisfied', value = '1', inplace = True)
final_data['JobSatisfaction'].replace(to_replace = '10 - Highly Satisfied', value = '10', inplace = True)
final_data['JobSatisfaction'].replace(to_replace = 'I prefer not to share', value = None, inplace = True)
final_data['JobSatisfaction'] = pd.to_numeric(final_data['JobSatisfaction'])
#final_data['JobSatisfaction'].describe()

In [193]:
#make compensation a meaningful variable
final_data['SalaryChange'].replace(to_replace = 'I do not want to share information about my salary/compensation', value = '', inplace = True)

#making employer size changes into a 5 point Likurt scale
#final_data['EmployerSizeChange'].replace(to_replace = 'Increased significantly', value = '5', inplace = True)
#final_data['EmployerSizeChange'].replace(to_replace = 'Increased slightly', value = '4', inplace = True)
#final_data['EmployerSizeChange'].replace(to_replace = 'Stayed the same', value = '3', inplace = True)
#final_data['EmployerSizeChange'].replace(to_replace = 'Decreased slightly', value = '2', inplace = True)
#final_data['EmployerSizeChange'].replace(to_replace = 'Decreased significantly', value = '1', inplace = True)
#final_data['EmployerSizeChange'] =  pd.to_numeric(final_data['EmployerSizeChange'])


In [194]:
#changing salary change information into a Likurt scale 
#final_data['SalaryChange'].replace(to_replace = 'Has increased 20% or more', value = '5', inplace = True)
#final_data['SalaryChange'].replace(to_replace = 'Has increased between 6% and 19%', value = '4', inplace = True)
#final_data['SalaryChange'].replace(to_replace = 'Has stayed about the same (has not increased or decreased more than 5%)', value = '3', inplace = True)
#final_data['SalaryChange'].replace(to_replace = 'Has decreased between 6% and 19%', value = '2', inplace = True)
#final_data['SalaryChange'].replace(to_replace = 'Has decreased 20% or more', value = '1', inplace = True)
#final_data['SalaryChange'].replace(to_replace = ['I was not employed 3 years ago', 'Other'], value = None, inplace = True)
#final_data['SalaryChange'] =  pd.to_numeric(final_data['SalaryChange'])


In [195]:
#changing title fit information into a Likurt scale 
#final_data['TitleFit'].replace(to_replace = 'Perfectly', value = '3', inplace = True)
#final_data['TitleFit'].replace(to_replace = 'Fine', value = '2', inplace = True)
#final_data['TitleFit'].replace(to_replace = 'Poorly', value = '1', inplace = True)
#final_data['TitleFit'] =  pd.to_numeric(final_data['TitleFit'])


In [196]:
#combine non-binary gender categories as they are sparse 
final_data['GenderSelect'].replace(to_replace = 'Non-binary, genderqueer, or gender non-conforming', value = 'A different identity', inplace = True)

In [197]:
#converting binary variables into strings so they are processed correctly as categories later on in modeling 
temp = final_data.max()
dftemp = temp.to_frame('max')
binaries = dftemp[dftemp['max'] == 1].index
#final_data[binaries].apply(str)
for i in binaries:
    final_data[i] = final_data[i].apply(str)
    final_data[i].replace(["0.0", '1.0'], ["No", "Yes"], inplace=True)

In [198]:
#fill NAs with the mean value so the rows don't get dropped by potential future classifiers 
final_data = final_data.fillna(final_data.mean())


In [199]:
#fill anything missing with a Missing value to clearly designate it 
#final_data = final_data.fillna('Missing')
final_data['Continent']= final_data['Continent'].str.replace( 'Missing', 'Other')

#final_data.isnull().sum()

In [200]:
#too related to job satisfaction which is the dependent variable later on so dropping it 
final_data = final_data.drop('TitleFit', axis=1)

In [201]:
final_data.shape

(5617, 277)

In [202]:
final_data['AlgorithmUnderstandingLevel'].unique()
final_data['AlgorithmUnderstandingLevel'] = final_data['AlgorithmUnderstandingLevel'].replace(
    ['Enough to code it from scratch and it will run blazingly fast and be super efficient', 
     'Enough to code it again from scratch, albeit it may run slowly',
    'Enough to refine and innovate on the algorithm',
     'Enough to explain the algorithm to someone non-technical',
     'Enough to tune the parameters properly',
     'Enough to run the code / standard library'
    ], ['Expert', 'Expert', 'Expert', 'Competent', 'Beginner', 'Beginner'])
final_data['AlgorithmUnderstandingLevel'].unique()

array(['Expert', 'Competent', 'Beginner', nan], dtype=object)

In [203]:
#write data out for example notebook
final_data.to_csv('./datasets/final_data.csv', index=False)